#### **NEM Review contract co-design workshop**

# Contract financial performance modelling

## Prepare environment

In [1]:
# Data handling
import geopandas as gpd
import numpy as np
import os
import pandas as pd

# Visualisation
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

# Custom functions
from scripts.common_functions import python_setup, get_mms_data, save_figure

# Set up Python
working_dir, charts_dir, data_dir = python_setup(2)

## Prepare data

### Access generators from latest Generation Information

In [2]:
# Check if generator details exist
gen_info_exist = os.path.exists(os.path.join(data_dir, "gen_info.csv"))

if not gen_info_exist:
    print("Generator information not found. Building dataset ...")

    # Check if source Excel file exists
    excel_path = os.path.join(data_dir, "NEM Generation Information July 2025.xlsx")
    if not os.path.exists(excel_path):
        raise FileNotFoundError(f"AEMO source file not found at: {excel_path}")

    try:
        # Load generator details
        gen_info = pd.read_excel(
            excel_path,
            sheet_name=4,
            usecols=["Region", "Asset Type", "Site Name", "Fuel Type", "DUID"],
            header=1,
        ).rename(columns={
            "Asset Type": "Existing",
            "Site Name": "Name",
            "Fuel Type": "Technology"})
    
        # Filter gen info
        gen_info = gen_info[
            gen_info["Region"].isin(["NSW1", "QLD1", "SA1", "VIC1"]) &
            (gen_info["Existing"] == "Existing Plant") &
            gen_info["Technology"].isin(["Solar - Solar", "Wind - Wind"])
        ].iloc[:, [0, 4, 2, 3]]  # Select only needed columns

        # Format gen info
        gen_info = gen_info.replace({
            "Region": {"NSW1": "NSW", "QLD1": "QLD", "SA1": "SA", "VIC1": "VIC"},
            "Technology": {"Solar - Solar": "Solar", "Wind - Wind": "Wind"}
        })
        gen_info["DUID"] = gen_info["DUID"].str.strip().str.upper()

        # Drop duplicate DUIDs
        initial_count = len(gen_info)
        gen_info.drop_duplicates(subset="DUID", inplace=True)
        gen_info.dropna(subset=["DUID"], inplace=True)
        gen_info = gen_info[gen_info["DUID"] != "-"]
        final_count = len(gen_info)
        if initial_count != final_count:
            print(f"Warning: Dropped {initial_count - final_count} duplicate and null DUIDs")

        # Set index and check for empty DataFrame
        gen_info.set_index("DUID", inplace=True)
        if gen_info.empty:
            raise ValueError("No generator information found after filtering")

        print(f"Loaded {len(gen_info)} generators from AEMO file.")

    except Exception as e:
        print(f"Error loading generator information: {e}")
        raise

else:
    print("Generator information found. Dataset loaded.")
    try:
        gen_info = pd.read_csv(
            os.path.join(data_dir, "gen_info.csv"),
            index_col="DUID"
        )
        print(f"Loaded {len(gen_info)} generators from existing file.")
    except Exception as e:
        print(f"Error loading saved generator information: {e}")
        raise

# Basic validation
if 'gen_info' not in locals() or gen_info.empty:
    raise ValueError("No generator information available")

# Display basic info
print(f"\nGenerator dataset summary:")
print(f"- Regions: {gen_info['Region'].unique()}")
print(f"- Technologies: {gen_info['Technology'].unique()}")
print(f"- Total generators: {len(gen_info)}")

Generator information found. Dataset loaded.
Loaded 178 generators from existing file.

Generator dataset summary:
- Regions: ['SA' 'VIC' 'NSW' 'QLD']
- Technologies: ['Solar' 'Wind']
- Total generators: 178


### Collect historic DUID data

If no data yet exists, execute SQL query to collect output, availability and maximum capacity data for each DUID.

In [3]:
# Check if DUID data exists
duid_data_exist = os.path.exists(os.path.join(data_dir, "duid_data", "duid_data.csv"))

if not duid_data_exist:
    # Initialise an empty list to collect all region data
    all_region_data = []

    # Process in batches by region
    batch_size = 10
    for region in gen_info["Region"].unique():
        region_data = []
        region_duids = gen_info[gen_info["Region"] == region].index.tolist()
        duid_batches = [region_duids[i:i + batch_size] for i in range(0, len(region_duids), batch_size)]

        # Collect DUID data from MMS
        for i, batch in enumerate(duid_batches):
            print(f"Processing {region} batch {i + 1} of {len(duid_batches)}", end="\r")
            df = get_mms_data(
                "collect_duid_data",
                arguments={
                    "start_date": "2020-01-01",
                    "end_date": "2024-12-31",
                    "duid_list": ",".join([f"'{d}'" for d in batch])})
            if df is not None:  # Check if query returned data
                region_data.append(df)

        print(f"Processing {region} batch data ... complete.")
        # Concatenate all batches for this region
        if region_data:  # Only concatenate if we got data
            region_df = pd.concat(region_data)
            all_region_data.append(region_df)

    # Combine all region data
    if all_region_data:  # Only proceed if we got data
        duid_data = pd.concat(all_region_data)

        # Filter out empty DUIDs (DUIDs with no data)
        valid_duids = duid_data["DUID"].unique()
        gen_info = gen_info.loc[valid_duids]

        # Add maximum capacity values (latest value for each DUID)
        def get_latest_capacity(duid):
            duid_records = duid_data[duid_data["DUID"] == duid]
            if not duid_records.empty:
                return duid_records.sort_values("Interval").iloc[-1]["Maximum capacity"]
            return np.nan

        gen_info["Maximum capacity"] = gen_info.index.map(get_latest_capacity)

        # Add generator start time flag
        def is_late_starter(duid):
            duid_records = duid_data[duid_data["DUID"] == duid]
            if duid_records.empty:
                return True
            return pd.isna(duid_records.iloc[0]["Maximum capacity"])

        gen_info["Late start"] = gen_info.index.map(is_late_starter)

        # Save amended gen info
        print("Saving generator information ...", end="\r")
        os.makedirs(data_dir, exist_ok=True)  # Ensure directory exists
        gen_info.to_csv(os.path.join(data_dir, "gen_info.csv"))
        print("Saving generator information ... complete.")

        # Save DUID data
        print("Saving DUID data ...", end="\r")
        os.makedirs(os.path.join(data_dir, "duid_data"), exist_ok=True)  # Ensure directory exists
        duid_data.to_csv(os.path.join(data_dir, "duid_data", "duid_data.csv"), index=False)
        print("Saving DUID data ... complete.")

        # Save individual DUID files
        print("Saving individual DUID data ...", end="\r")
        for duid in duid_data["DUID"].unique():
            region = gen_info.at[duid, "Region"]
            duid_df = duid_data[duid_data["DUID"] == duid].drop(columns="DUID")
            duid_df.to_csv(os.path.join(data_dir, "duid_data", f"{region}_{duid}.csv"), index=False)
        print("Saving individual DUID data ... complete.")

        # Save region DUID files
        print("Saving region DUID data ...", end="\r")
        for region in gen_info["Region"].unique():
            region_duids = gen_info[gen_info["Region"] == region].index.tolist()
            region_duid_data = duid_data[duid_data["DUID"].isin(region_duids)]
            region_duid_data.to_csv(os.path.join(data_dir, "duid_data", f"duid_data_{region}.csv"), index=False)
        print("Saving region DUID data ... complete.")

else:
    print("Loading DUID generation data ...", end="\r")
    try:
        duid_data = pd.read_csv(
            os.path.join(data_dir, "duid_data", "duid_data.csv"))
        print(f"Loaded DUID generation data from existing file.")
    except Exception as e:
        print(f"Error loading saved generator information: {e}")
        raise

# Count DUIDs used in analysis
if "duid_data" in locals():  # Only if we collected data
    duid_counts = gen_info.reset_index().pivot_table(
        index="Region",
        columns="Technology",
        values="DUID",
        aggfunc="count",
        fill_value=0).astype(int)
    print("\nDUID counts by region and technology:")
    print(duid_counts)

Loaded DUID generation data from existing file.

DUID counts by region and technology:
Technology  Solar  Wind
Region                 
NSW            36    18
QLD            34     6
SA             16    26
VIC            14    28


### Collect whole-of-market data

If no data yet exists, execute SQL query to collect SA prices and total demand data.

In [5]:
# Demand and prices path
demand_and_prices_path = os.path.join(data_dir, "demand_and_prices.csv")

# Check if file exists
if not os.path.exists(demand_and_prices_path):
    print("demand_and_prices.csv not found. Querying database ...")

    # Get generation and price data
    demand_and_prices = get_mms_data(
        script_name="collect_demand_and_prices",
        arguments={
            "start_date": "2020-01-01",
            "end_date": "2024-12-31"})
    demand_and_prices.set_index("Interval", inplace=True)

    # Save data to file
    demand_and_prices.to_csv(demand_and_prices_path)
    print(f"Data saved.")

else:
    print(f"Loading existing demand_and_prices.csv file ...")
    # Load existing data
    demand_and_prices = pd.read_csv(
        demand_and_prices_path,
        parse_dates=[0],
        index_col=0)
    print("Data loaded.")

Loading existing demand_and_prices.csv file ...
Data loaded.


## Wind and solar market observations

Calculate and visualise metrics of:
* Size of the wind and solar fleets over time (2020 to 2024)
* Locations of generators
* Capacity factors of generators
* Seasonal shape of generation and load
* Intraday shape of generation and load
* Demand-weighted prices for each region by wind, solar and region-wide

### Size of wind and solar fleets

In [ ]:
# Prepare data (aggregate to monthly)
# Attached 'technology' to DUID data
sa_duid_data["Technology"] = sa_gen_info.loc[
    sa_duid_data["DUID"],
    "Technology"].values

# Aggregate to monthly
chart_data = sa_duid_data.pivot_table(
    index="Interval",
    columns="Technology",
    values="Maximum capacity",
    aggfunc="sum").resample("ME").mean()

# Save data to file
chart_data.to_csv(
    os.path.join(data_dir, "growth_of_sa_fleet.csv"))

# Plot fleet sizes over time
fig, ax = plt.subplots(
    figsize=(12, 6.75),
    tight_layout=True)
for i, tech in enumerate(["Wind", "Solar"]):
    ax.step(
        x=chart_data.index,
        y=chart_data[tech],
        where="pre",
        linewidth=3,
        color=f"C{i}",
        label=tech,
        linestyle=["-", "--"][i])
plt.xlabel(None)
plt.ylabel("Capacity (MW)")
ax.yaxis.set_major_formatter(lambda x, p: f"{x:,.0f}")
plt.ylim(0)
plt.legend()
for frame in ["top", "right", "bottom", "left"]:
    ax.spines[frame].set_visible(False)
plt.show()
save_figure(fig, "growth_of_sa_fleet", charts_dir)

### Generator capacity factors

In [ ]:
# Calculate CFs
cf_pivot = sa_duid_data.pivot_table(
    index="DUID",
    columns="Technology",
    values="Relative output",
    aggfunc="mean")
capacity_factors = cf_pivot.reset_index().melt(
    id_vars="DUID",
    var_name="Technology",
    value_name="Capacity factor").dropna()
# Save CFs
capacity_factors.to_csv(
    os.path.join(data_dir, "sa_capacity_factors.csv"),
    index=False)

# Plot CFs by technology
fig, ax = plt.subplots(
    figsize=(12, 6.75),
    tight_layout=True)
sns.boxplot(
    data=capacity_factors,
    x="Capacity factor",
    y="Technology",
    hue="Technology",
    palette=["C1", "C0"],
    ax=ax,
    linewidth=3,
    fliersize=15)
ax.set_ylabel(None)
ax.xaxis.set_major_formatter(lambda x, p: f"{x:.0%}")
for frame in ["top", "right", "bottom", "left"]:
    ax.spines[frame].set_visible(False)
plt.show()
save_figure(fig, "sa_fleet_capacity_factors", charts_dir)

# Calculate aggregate CF
fleet_cf = sa_duid_data.pivot_table(
    index="Technology",
    values=["Output", "Maximum capacity"],
    aggfunc="sum")
fleet_cf["Capacity factor"] = fleet_cf["Output"] / fleet_cf["Maximum capacity"]
print("Aggregate fleet capacity factor:")
for tech in fleet_cf.index:
    print(f"{tech}: {fleet_cf.at[tech, 'Capacity factor']:.1%}")

### Generator locations

In [ ]:
# Load states shapefile
states = gpd.read_file(
    os.path.join(data_dir, "australian_states_map.zip"))

# Convert DUIDs to GeoDataFrame
sa_gen_geo = gpd.GeoDataFrame(
    sa_gen_info,
    geometry=gpd.points_from_xy(
        sa_gen_info.Longitude,
        sa_gen_info.Latitude,
        crs=states.crs))

# Plot states outline
marker_sizes = np.sqrt(sa_gen_geo["Maximum capacity"] / np.pi)
fig, ax = plt.subplots(
    figsize=(12, 12),
    tight_layout=True)
sa_gen_geo.plot(
    "Technology",
    cmap=mpl.colors.ListedColormap(["C1", "C0"]),
    markersize=(
        (np.sqrt(sa_gen_geo["Maximum capacity"] / np.pi) - marker_sizes.min()) / 
        (marker_sizes.max() - marker_sizes.min()) * 500 + 25),
    alpha=.75,
    edgecolor="white",
    ax=ax,
    legend=True)
limits = ax.get_xlim(), ax.get_ylim()
states.boundary.plot(
    color="silver",
    linestyle="--",
    ax=ax)
plt.xlim(limits[0][0], limits[0][1])
plt.ylim(limits[1][0], limits[1][1])
ax.set_axis_off()
plt.show()
save_figure(fig, "sa_generator_locations", charts_dir)

### Seasonal shape of generation and load

In [ ]:
# Monthly and quarterly generation data
gen_data = sa_duid_data.pivot_table(
    index="Interval",
    columns="Technology",
    values="Output",
    aggfunc="sum")
gen_mth = gen_data.resample("ME").mean()
gen_mth.index.name = "Month"
gen_qtr = gen_data.resample("QE").mean()
gen_qtr.index.name = "Quarter"
# Save generation data
gen_mth.to_csv(
    os.path.join(data_dir, "sa_monthly_generation.csv"))
gen_qtr.to_csv(
    os.path.join(data_dir, "sa_quarterly_generation.csv"))

# Plot generation
fig, ax = plt.subplots(
    figsize=(12, 6.75),
    tight_layout=True)
for i, tech in enumerate(["Wind", "Solar"]):
    ax.step(
        x=gen_mth.index,
        y=gen_mth[tech],
        where="pre",
        linewidth=3,
        color=f"C{i}",
        label=tech)
plt.ylabel("MW")
plt.ylim(0)
plt.legend()
ax.yaxis.set_major_formatter(lambda x, p: f"{x:,.0f}")
for frame in ["top", "right", "bottom", "left"]:
    ax.spines[frame].set_visible(False)
plt.show()
save_figure(fig, "sa_mean_monthly_generation", charts_dir)

fig, ax = plt.subplots(
    figsize=(12, 6.75),
    tight_layout=True)
for i, tech in enumerate(["Wind", "Solar"]):
    ax.step(
        x=gen_qtr.index,
        y=gen_qtr[tech],
        where="pre",
        linewidth=3,
        color=f"C{i}",
        label=tech)
plt.ylabel("MW")
plt.ylim(0)
plt.legend()
ax.yaxis.set_major_formatter(lambda x, p: f"{x:,.0f}")
for frame in ["top", "right", "bottom", "left"]:
    ax.spines[frame].set_visible(False)
plt.show()
save_figure(fig, "sa_mean_quarterly_generation", charts_dir)

In [ ]:
# Monthly and quarterly load data
load_mth = sa_demand_and_prices.resample("ME")["Total demand"].mean()
load_mth.index.name = "Month"
load_qtr = sa_demand_and_prices.resample("QE")["Total demand"].mean()
load_qtr.index.name = "Quarter"
# Save load data
load_mth.to_csv(
    os.path.join(data_dir, "sa_monthly_load.csv"))
load_qtr.to_csv(
    os.path.join(data_dir, "sa_quarterly_load.csv"))

# Plot load
fig, ax = plt.subplots(
    figsize=(12, 6.75),
    tight_layout=True)
ax.step(
    x=load_mth.index,
    y=load_mth,
    where="pre",
    linewidth=3,
    color="C2")
plt.ylabel("MW")
plt.ylim(0)
ax.yaxis.set_major_formatter(lambda x, p: f"{x:,.0f}")
for frame in ["top", "right", "bottom", "left"]:
    ax.spines[frame].set_visible(False)
plt.show()
save_figure(fig, "sa_mean_monthly_load", charts_dir)

fig, ax = plt.subplots(
    figsize=(12, 6.75),
    tight_layout=True)
ax.step(
    x=load_qtr.index,
    y=load_qtr,
    where="pre",
    linewidth=3,
    color="C2")
plt.ylabel("MW")
plt.ylim(0)
ax.yaxis.set_major_formatter(lambda x, p: f"{x:,.0f}")
for frame in ["top", "right", "bottom", "left"]:
    ax.spines[frame].set_visible(False)
plt.show()
save_figure(fig, "sa_mean_quarterly_load", charts_dir)

### Intraday shape of generation and load

In [ ]:
# Prepare generation data
sa_intraday_gen_data = sa_duid_data.pivot_table(
    index="Interval",
    columns="Technology",
    values="Output",
    aggfunc="sum")
sa_intraday_gen_data["30-min interval"] = sa_intraday_gen_data.index.hour * 2 + sa_intraday_gen_data.index.minute // 30
chart_data = sa_intraday_gen_data.pivot_table(
    index="30-min interval",
    values=["Wind", "Solar"],
    aggfunc="mean")

# Save intraday generation data
chart_data.to_csv(
    os.path.join(data_dir, "sa_mean_intraday_generation.csv"))

# Plot intraday generation
fig, ax = plt.subplots(
    figsize=(12, 6.75),
    tight_layout=True)
for i, tech in enumerate(["Wind", "Solar"]):
    ax.step(
        x=chart_data.index,
        y=chart_data[tech],
        where="pre",
        linewidth=3,
        color=f"C{i}",
        label=tech)
plt.ylabel("MW")
plt.xlabel("Time of day")
plt.ylim(0)
plt.legend()
ax.yaxis.set_major_formatter(lambda x, p: f"{x:,.0f}")
ax.xaxis.set_major_formatter(lambda x, p: f"{pd.to_datetime('2025-01-01') + pd.DateOffset(minutes=x * 30):%H:%M}")
for frame in ["top", "right", "bottom", "left"]:
    ax.spines[frame].set_visible(False)
plt.show()
save_figure(fig, "sa_mean_intraday_generation", charts_dir)

In [ ]:
# Prepare load data
sa_demand_and_prices["30-min interval"] = sa_demand_and_prices.index.hour * 2 + sa_demand_and_prices.index.minute // 30
chart_data = sa_demand_and_prices.pivot_table(
    index="30-min interval",
    values="Total demand",
    aggfunc="mean")

# Save intraday generation data
chart_data.to_csv(
    os.path.join(data_dir, "sa_mean_intraday_load.csv"))

# Plot intraday generation
fig, ax = plt.subplots(
    figsize=(12, 6.75),
    tight_layout=True)
ax.step(
    x=chart_data.index,
    y=chart_data["Total demand"],
    where="pre",
    linewidth=3,
    color="C2")
plt.ylabel("MW")
plt.xlabel("Time of day")
plt.ylim(0)
ax.yaxis.set_major_formatter(lambda x, p: f"{x:,.0f}")
ax.xaxis.set_major_formatter(lambda x, p: f"{pd.to_datetime('2025-01-01') + pd.DateOffset(minutes=x * 30):%H:%M}")
for frame in ["top", "right", "bottom", "left"]:
    ax.spines[frame].set_visible(False)
plt.show()
save_figure(fig, "sa_mean_intraday_load", charts_dir)

### Volume-weighted average prices: SA market, wind and solar

In [ ]:
# Prepare volume-weighted price data
sa_demand_and_prices["Revenue"] = sa_demand_and_prices["Total demand"] * sa_demand_and_prices["Price"]

# Merge price and generation data
sa_gen_prices = sa_duid_data.pivot_table(
    index="Interval",
    columns="Technology",
    values="Output",
    aggfunc="sum")
sa_gen_prices["Price"] = sa_demand_and_prices.loc[
    sa_gen_prices.index, "Price"]
sa_gen_prices["Solar revenue"] = sa_gen_prices["Solar"] * sa_gen_prices["Price"]
sa_gen_prices["Wind revenue"] = sa_gen_prices["Wind"] * sa_gen_prices["Price"]

# Calculate monthly prices
dwa_prices_sa_mth = sa_demand_and_prices.resample("ME")[["Total demand", "Revenue"]].sum()
dwa_prices_sa_solar_mth = sa_gen_prices.resample("ME")[["Solar", "Solar revenue"]].sum()
dwa_prices_sa_wind_mth = sa_gen_prices.resample("ME")[["Wind", "Wind revenue"]].sum()
dwa_prices_sa_mth["DWA price"] = dwa_prices_sa_mth["Revenue"] / dwa_prices_sa_mth["Total demand"]
dwa_prices_sa_solar_mth["DWA price"] = dwa_prices_sa_solar_mth["Solar revenue"] / dwa_prices_sa_solar_mth["Solar"]
dwa_prices_sa_wind_mth["DWA price"] = dwa_prices_sa_wind_mth["Wind revenue"] / dwa_prices_sa_wind_mth["Wind"]
# Calculate quarterly prices
dwa_prices_sa_qtr = sa_demand_and_prices.resample("QE")[["Total demand", "Revenue"]].sum()
dwa_prices_sa_solar_qtr = sa_gen_prices.resample("QE")[["Solar", "Solar revenue"]].sum()
dwa_prices_sa_wind_qtr = sa_gen_prices.resample("QE")[["Wind", "Wind revenue"]].sum()
dwa_prices_sa_qtr["DWA price"] = dwa_prices_sa_qtr["Revenue"] / dwa_prices_sa_qtr["Total demand"]
dwa_prices_sa_solar_qtr["DWA price"] = dwa_prices_sa_solar_qtr["Solar revenue"] / dwa_prices_sa_solar_qtr["Solar"]
dwa_prices_sa_wind_qtr["DWA price"] = dwa_prices_sa_wind_qtr["Wind revenue"] / dwa_prices_sa_wind_qtr["Wind"]
# Save to file
dwa_prices_sa_mth.iloc[:, -1:].to_csv(
    os.path.join(data_dir, "sa_dwa_prices_monthly.csv"))
dwa_prices_sa_solar_mth.iloc[:, -1:].to_csv(
    os.path.join(data_dir, "sa_dwa_solar_prices_monthly.csv"))
dwa_prices_sa_wind_mth.iloc[:, -1:].to_csv(
    os.path.join(data_dir, "sa_dwa_wind_prices_monthly.csv"))
dwa_prices_sa_qtr.iloc[:, -1:].to_csv(
    os.path.join(data_dir, "sa_dwa_prices_quarterly.csv"))
dwa_prices_sa_solar_qtr.iloc[:, -1:].to_csv(
    os.path.join(data_dir, "sa_dwa_solar_prices_quarterly.csv"))
dwa_prices_sa_wind_qtr.iloc[:, -1:].to_csv(
    os.path.join(data_dir, "sa_dwa_wind_prices_quarterly.csv"))

# Plot load
fig, ax = plt.subplots(
    figsize=(12, 6.75),
    tight_layout=True)
ax.step(
    x=dwa_prices_sa_wind_mth.index,
    y=dwa_prices_sa_wind_mth["DWA price"],
    where="pre",
    linewidth=3,
    color="C0",
    label="Wind")
ax.step(
    x=dwa_prices_sa_solar_mth.index,
    y=dwa_prices_sa_solar_mth["DWA price"],
    where="pre",
    linewidth=3,
    color="C1",
    label="Solar")
ax.step(
    x=dwa_prices_sa_mth.index,
    y=dwa_prices_sa_mth["DWA price"],
    where="pre",
    linewidth=3,
    color="C2",
    label="SA market")
plt.ylabel("$/MWh")
plt.legend()
ax.yaxis.set_major_formatter(lambda x, p: f"{x:,.0f}")
for frame in ["top", "right", "bottom", "left"]:
    ax.spines[frame].set_visible(False)
plt.show()
save_figure(fig, "dwa_prices_sa_monthly", charts_dir)

# Plot load
fig, ax = plt.subplots(
    figsize=(12, 6.75),
    tight_layout=True)
ax.step(
    x=dwa_prices_sa_qtr.index,
    y=dwa_prices_sa_qtr["DWA price"],
    where="pre",
    linewidth=3,
    color="C2",
    label="SA market")
ax.step(
    x=dwa_prices_sa_wind_qtr.index,
    y=dwa_prices_sa_wind_qtr["DWA price"],
    where="pre",
    linewidth=3,
    color="C0",
    label="Wind")
ax.step(
    x=dwa_prices_sa_solar_qtr.index,
    y=dwa_prices_sa_solar_qtr["DWA price"],
    where="pre",
    linewidth=3,
    color="C1",
    label="Solar")
plt.ylabel("$/MWh")
plt.legend()
ax.yaxis.set_major_formatter(lambda x, p: f"{x:,.0f}")
for frame in ["top", "right", "bottom", "left"]:
    ax.spines[frame].set_visible(False)
plt.show()
save_figure(fig, "dwa_prices_sa_quarterly", charts_dir)